## Step-1:  Importing required libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io

## Step-2:  Loading the image


In [2]:
img = cv2.imread('dali.jpg')
# img = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)

In [3]:
# cv2.imshow('original',img)

cv2.waitKey(0)
cv2.destroyAllWindows() 

## Step-3: Converting an image into gray_scale image
### Converting an image into grayscale gives us black & white pixels in the image which is used for creating a pencil sketch.

In [4]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-9d_dfo3_\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
cv2.imshow('grayscale',img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows() 

## Step-4: Inverting the image
### We are using the bitwise_not function which is used to make brighter regions lighter and vice versa so that we can find the edges to create a pencil sketch. 

In [ ]:
img_invert = cv2.bitwise_not(img_gray)

In [ ]:
cv2.imshow('inv',img_invert)
cv2.waitKey(0)
cv2.destroyAllWindows() 

## Step-5: Smoothing the image 
### We have used the gaussian blur technique with 21 x 21 pixel and the default sigma values filter on the image to smoothen our image. By increasing the filter size, we can create thin lines for our sketch and it is used to reduce the noise in the image. 

In [ ]:
img_smoothing = cv2.GaussianBlur(img_invert, (21, 21),sigmaX=0, sigmaY=0)

cv2.imshow('smoothen',img_smoothing)
cv2.waitKey(0)
cv2.destroyAllWindows() 

## Final Output as Pencil Sketch

In [5]:
def dodgeV2(x, y):
    return cv2.divide(x, 255 - y, scale=256)

def burnV2(image, mask):
    return 255 -cv2.divide(255-image, 255-mask, scale=256)

In [6]:
dodge_img = dodgeV2(img_gray, img_smoothing)

NameError: name 'img_gray' is not defined

In [ ]:
# cv2.imshow('sketch',dodge_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows() 

In [ ]:
# burned_img = dodgeV2(dodge_img, img_smoothing)

In [ ]:
cv2.imshow('sketch',burned_img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

In [7]:
import datetime
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import base64
from PIL import Image

import dash_bootstrap_components as dbc


app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div([
    html.H1("SKETCH YOUR IMAGE!!",className='title'),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-image-upload'),
    
    dbc.Button("Submit", id='submit-val',color="success",size="lg"),
    html.Div(id='output-sketch-upload'),
])

@app.callback(Output('output-sketch-upload','children'),
            [Input('submit-val', 'n_clicks')])

def sketch_pic(n_clicks):
    if n_clicks:
        name = ''.join(name_img.split('.')[0]) + '.png'
        image = base64.b64decode(str(contents_img[22:])) 
        imagePath = ("test.jpeg")
        img = Image.open(io.BytesIO(image))
        img.save(name,'png')

        image_file = cv2.imread(name)

        img_gray = cv2.cvtColor(image_file, cv2.COLOR_BGR2GRAY)

        img_invert = cv2.bitwise_not(img_gray)

        img_smoothing = cv2.GaussianBlur(img_invert, (21, 21),sigmaX=0, sigmaY=0)

        final_img = dodgeV2(img_gray, img_smoothing)

        cv2.imwrite('result.png',final_img)

        test_png = 'result.png'
        test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')

        return html.Div([
            html.H5(name+' Sketching!!'),
            # HTML images accept base64 encoded strings in the same format
            # that is supplied by the upload
            html.Img(src='data:image/png;base64,{}'.format(test_base64),style={'height':'50%', 'width':'50%'}),
            html.Hr(),
        ])


contents_img=0
name_img=0
def parse_contents(contents, filename, date):
    global contents_img, name_img
    name_img=filename
    contents_img = contents
    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        # HTML images accept base64 encoded strings in the same format
        # that is supplied by the upload
        html.Img(src=contents,style={'height':'50%', 'width':'50%'}),
        html.Hr(),
    ])


@app.callback(Output('output-image-upload', 'children'),
              [Input('upload-image', 'contents')],
              [State('upload-image', 'filename'),
               State('upload-image', 'last_modified')])
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /assets/style.css?m=1595541614.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_1m1600775263.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_1m1600775263.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_1m1600775263.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_1m1600775268.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_1m1600775263.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2020 19:09:35] "GET /_dash-component-suites/dash_bootstrap_components